In [ ]:
# imports
import requests
import json
import pandas as pd

from bs4 import BeautifulSoup


# inicializando um dataframe
df = pd.DataFrame(data = {'nome':[], 'endereco': [], 'cep': [], 'telefone': [], 'telefone_adicional': [], 'cidade': [], 'estado': []})

# pegando a página de busca para pegar a lista de estados
formulario = requests.get('https://www.vizzano.com.br/encontre/pt/lojas-fisicas') # chamada da página de busca de lojas
formulario = BeautifulSoup(formulario.text, 'html.parser') # parseando o texto para html, facilita a busca

# inicializando a lista de estados
estados = []

# passando por cada item de estado e pegando o valor dele para adicionar na lista
for estado in formulario.select('#estado > option'):
    if estado.get('value'):
        estados.append(estado.get('value'))

i = 0 # contador para saber no terminal quantos já foram

# percorrendo a lista de estados
for estado in estados:
    # pegando a lista de cidades do estado
    payload={'secao': 'onde', 'pais': 'BRASIL', 'estado': estado, 'cidade': ''}
    cidades = json.loads(requests.post("https://www.vizzano.com.br/acoes/lista-cidades", data=payload).content.decode()) # lista de cidades em json
    
    # percorrendo a lista de cidades
    for cidade in cidades:
        # pegando a lista de lojas na cidade
        payload['cidade'] = cidade.get('label')
        lojas = requests.post("https://www.vizzano.com.br/acoes/lista-lojas", data=payload).content.decode() # lista de lojas em html
        soup = BeautifulSoup(lojas, 'html.parser') # parceando o texto para html, facilista a busca
        lojas = soup.find_all('li') # encontra todas as lojas da cidade
        # percorrendo a lista de lojas
        for loja in lojas:
            linhas = loja.article.find_all('p') # encontrando as informações da loja
            # tenta encontrar números de telefone adicionais
            try:
                numeros_adicionais = linhas[3].find_all('span')
                numeros = []
                for numero_adicional in numeros_adicionais:
                    numeros.append(numero_adicional.contents[0])
                numero_adicional =  ','.join(numeros)
            except:
                numero_adicional = None
            telefone_adicional = numero_adicional
            
            # adiciona no dataframe as informações da loja
            df = df.append(
                {
                    'nome': loja.h6.span.contents[0],
                    'endereco': linhas[0].contents[0],
                    'cep': linhas[1].contents[0],
                    'telefone': linhas[2].strong.contents[0],
                    'telefone_adicional': telefone_adicional,
                    'cidade': cidade.get('label'),
                    'estado': estado
                },
                ignore_index=True
            )
            
            i = i + 1
            print(f'{i} - {estado} - {cidade.get("label")}')

df.to_csv('vizzano.csv') # converte para csv
df.to_excel('vizzano.xlsx') # converte para excel